# Adaptive robust lot sizing

This tutorial shows how to construct and solve the Adaptive robust lot sizing optimization problem. The problem can be formulated as follows:
$$
\begin{align}
\min_{\mathbf{x},\mathbf{Y}}\; &\mathbf{c}^T\mathbf{x} + \sum_{i=1}^N\sum_{j=1}^N t_{i,j}y_{i,j}\nonumber \\

\text{s.t. } & \mathbf{d} \leq \sum_{i=1}^N \mathbf{y}_{i}^R - \sum_{i=1}^N \mathbf{y}_{i}^C + \mathbf{x} \nonumber \\

&\mathbf{0} \leq \mathbf{x}\leq \mathbf{k} \nonumber \\

&\mathbf{d}\in \left\{\mathbf{d}: \mathbf{0}\leq \mathbf{d}\leq \mathbf{d}_{max},\; \sum_{i=1}^N d_i \leq \Gamma\right\} \nonumber \\
\end{align}
$$

where $\mathbf{d}\in\mathbb{R}^N$ is the uncertain demand vector, $\mathbf{Y}\in\mathbb{R}^{N\times N}$ is the stock matrix, i.e. $y_{i,j}$ stock can be transported from location $i$ to location $j$ at cost $t_{i,j}$. $\mathbf{y}_i^R, \mathbf{y}_i^C$ denote the $i$-th row/column of $\mathbf{Y}$, respectively. The stock matrix $\mathbf{Y}$ depends on the demand $\mathbf{d}$ and can be written as
$$
\mathbf{Y} = \mathbf{Y^0} + \sum_{i=1}^{N}\mathbf{Y}_i^d d_i
$$
where $\mathbf{Y}^0, \left\{ \mathbf{Y}_i^d \right\}_i \in \mathbb{R}^{N\times N}$.

In [1]:
import cvxpy as cp
import numpy as np

from lropt import Ellipsoidal
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

We define the constants as shown below:

In [2]:
N = 30                                                           # Number of stores
c = 20                                                           # Storage cost
k = 20                                                           # Stock capacity
dmax = 20                                                        # Maximum demand
Gamma = 20*np.sqrt(N)                                            # Uncertainty budget
coordinates = 10*np.random.rand(2, N)                            # Stores coordinates
t = ((coordinates[[0]] - coordinates[[0]].T) ** 2
     + (coordinates[[1]] - coordinates[[1]].T) ** 2) ** 0.5      # Cost of transporting stocks

The problem is formulated and solved in the block below, using the Ellipsoidal uncertainty set which is defined as follows:

Ellipsoidal uncertainty: $\{u \mid \| Au + b \|_p \leq \rho \}$

- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.

We directly add the inequality constraints on $\eta$ using the $\textit{lb},\textit{ub}$ optional arguments, which read as follows:
$$
lb \leq \mathbf{u} \leq ub \nonumber \\
$$

Note that $\mathbf{Y}$ is defined indirectly through $\mathbf{Y}^0, \left\{ \mathbf{Y}_i^d \right\}_i$, and is a function of the demand vector $\mathbf{d}$.

In [3]:
uncertainty_set = Ellipsoidal(rho=Gamma, p=1, lb=0, ub=dmax)        # Ellipsoidal uncertainty set
x = cp.Variable(N)                                                  # Optimization variables, stock allocations
d = UncertainParameter(N, uncertainty_set=uncertainty_set)          # Uncertain parameters, demands
y_0 = cp.Variable((N,N))                                            # Helper optimization variable for y
y_d = [cp.Variable((N,N)) for _ in range(N)]                        # Helper optimization variables for y
y = 0                                                               # Optimization variable, adaptive stock transportation decision
#Define y as a wait-and-see decisision variable
for j in range(N):
    y += y_d[j]*d[j]
y += y_0

#cp.multiply is elementwise multiplication
objective = cp.Minimize(cp.sum(c*x) + cp.sum(cp.multiply(t,y)))     # Problem objective
constraints = [
                d <= cp.sum(y, axis=0) - cp.sum(y, axis=1) + x,
                y >= 0,
                x >= 0,
                x <= k,
              ]                                                     # Problem constraints
prob = RobustProblem(objective=objective, constraints=constraints)  # Define the problem using LROPT
prob.solve()                                                        # Solve the problem using LROPT

ValueError: DRP error: not able to process non multiplication/additions